# TF-IDF with Greek

Use this notebook to experiment with calculating TF-IDF with Greek. You won't be able to use scikit-learn's `"english"` stop words, and in general you will have to be in charge of a much larger part of the pipeline.

But this could be a useful exercise for those of you interested in the additional complexity of working with Greek!

In [2]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 47.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 54.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 53.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 55.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0
    Uninstalling python-da

In [3]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.read_pickle("./greek-tragedy-by-line.pickle")

df[df['speaker'].str.lower() == 'chorus']

,n,urn,dramatist,title,speaker,text
0,1,urn:cts:greekLit:tlg0085.tlg001.perseus-eng2,Aeschylus,Suppliant Maidens,Chorus,"May Zeus who guards suppliants look graciously upon our company, which boarded a ship and put to sea from the outlets of the fine sand of the"
1,5,urn:cts:greekLit:tlg0085.tlg001.perseus-eng2,Aeschylus,Suppliant Maidens,Chorus,whose pastures border
2,10,urn:cts:greekLit:tlg0085.tlg001.perseus-eng2,Aeschylus,Suppliant Maidens,Chorus,"It was Danaus, our father, adviser and leader, who, considering well our course, decided, as the best of all possible evils, that we flee with all speed over the waves of the sea"
3,15,urn:cts:greekLit:tlg0085.tlg001.perseus-eng2,Aeschylus,Suppliant Maidens,Chorus,and find a haven on
4,20,urn:cts:greekLit:tlg0085.tlg001.perseus-eng2,Aeschylus,Suppliant Maidens,Chorus,"could we come with these wool-wreathed branches in our hands, sole weapons of the suppliant? O realm, O land, and clear water; gods on high and earth-bound powers, grievous in your vengeance,"
...,...,...,...,...,...,...
17900,1306,urn:cts:greekLit:tlg0006.tlg003.perseus-eng2,Euripides,Medea,Chorus,"Unhappy man, thou knowest not the full extent of thy misery, else had thou never said those words."
17902,1309,urn:cts:greekLit:tlg0006.tlg003.perseus-eng2,Euripides,Medea,Chorus,Thy sons are dead; slain by their own mother’s hand.
17904,1311,urn:cts:greekLit:tlg0006.tlg003.perseus-eng2,Euripides,Medea,Chorus,Thy children are no more; be sure of this.
17906,1313,urn:cts:greekLit:tlg0006.tlg003.perseus-eng2,Euripides,Medea,Chorus,Throw wide the doors and see thy children’s murdered corpses.
